In [61]:
from bs4 import BeautifulSoup
import requests

In [62]:
player_base_odi_url = "http://www.howstat.com/cricket/Statistics/Players/PlayerOverview_ODI.asp?PlayerID="
player_base_t20_url = "http://www.howstat.com/cricket/Statistics/Players/PlayerOverview_T20.asp?PlayerID="
player_base_test_url = "http://www.howstat.com/cricket/Statistics/Players/PlayerOverview.asp?PlayerID="
player_base_ipl_url = "http://www.howstat.com/cricket/Statistics/IPL/PlayerOverview.asp?PlayerID="

URL = [player_base_odi_url, player_base_t20_url, player_base_test_url]

In [ ]:
player_id = '4675'

In [ ]:
request_url = player_base_odi_url + player_id
website_html = requests.get(request_url)
soup = BeautifulSoup(website_html.content, 'html.parser')

In [ ]:
def odi_t20_stat(soup):
    match_index_mapping = {
        "batting": {
            "innings": "Innings:",
            "notOuts": "Not Outs:",
            "agrregate": "Aggregate:",
            "highestScore": "Highest Score:",
            "average": "Average:",
            "50s": "50s:",
            "100s": "100s:",
            "ducks": "Ducks:",
            "4s": "4s:",
            "6s": "6s:",
            "scoringRate": "Scoring Rate",
            "BallsFaced": "Balls Faced:"
            },
        "bowling": {
            "ballDeliveries": "Balls:",
            "runsConceded": "Runs Conceded:",
            "wicktes": "Wickets:",
            "bowlingAverage": "Average:",
            "5Wickets": "5 Wickets in Innings:",
            "bestBowling": "Best:",
            "bowlingEconomyRate": "Economy Rate:",
            "bowlingStrikeRate": "Strike Rate:",
        },
        "captaincy": {
            "Matches/Won/Lost:": "Matches/Won/Lost:"
            }
    }

    stats = {
        "batting": {},
        "bowling": {},
        "captaincy": {}
    }

    for key1 in match_index_mapping.keys():
        for key2 in match_index_mapping[key1].keys():
            try:
                stats[key1][key2] = soup.find('span', string=match_index_mapping[key1][key2]).find_next('td').text.strip()
            except AttributeError:
                stats[key1][key2] = None

    return stats

In [ ]:

def test_stat(soup):
    match_index_mapping ={
        "batting": {
            "innings": "Innings:",
            "notOuts": "Not Outs:",
            "agrregate": "Aggregate:",
            "highestScore": "Highest Score:",
            "average": "Average:",
            "50s": "50s:",
            "100s": "100s:",
            "ducks": "Ducks:",
            "4s": "4s:",
            "6s": "6s:",
            "scoringRate": "Scoring Rate",
            "BallsFaced": "Balls Faced:"
            },
        "bowling": {
            "ballDeliveries": "Balls:",
            "runsConceded": "Runs Conceded:",
            "wicktes": "Wickets:",
            "bowlingAverage": "Average:",
            "5Wickets": "5 Wickets in Innings:",
            "bestBowling": "Best - Match:",
            "bowlingEconomyRate": "Economy Rate:",
            "bowlingStrikeRate": "Strike Rate:",
        },
        "captaincy": {
            "Matches/Won/Lost:": "Matches/Won/Lost:"
            }
    }

    stats = {
        "batting": {},
        "bowling": {},
        "captaincy": {}
    }

    for key1 in match_index_mapping.keys():
        for key2 in match_index_mapping[key1].keys():
            try:
                stats[key1][key2] = soup.find('span', string=match_index_mapping[key1][key2]).find_next('td').text.strip()
            except AttributeError:
                stats[key1][key2] = None

    return stats


In [40]:
COUNTRY_MAP = {
    "afghanistan": "AFG",
    "australia": "AUS",
    "bangladesh": "BAN",
    "england": "ENG",
    "india": "IND",
    "ireland": "IRE",
    "new zealand": "NZ",
    "pakistan": "PAK",
    "south africa": "SA",
    "sri lanka": "SL",
    "west indies": "WI",
    "zimbabwe": "ZIM",
    "netherlands": "NED"
}

In [63]:
def get_player_info(soup):
    try:
        player_info = {}
        personal_table = soup.find('table', {'border': '0', 'width': '100%', 'cellpadding': 3})
        all_tr = personal_table.find_all('tr')
        for tr in all_tr:
            if 'full name' in tr.find_next('td').text.lower():
                player_info['name'] = tr.find_next('td').find_next('td').text.strip()
            elif 'matches' in tr.find_next('td').text.lower():
                player_info['matches'] = tr.find_next('td').find_next('td').text.split('(')[0].strip()
            elif 'age' in tr.find_next('td').text.lower():
                player_info['age'] = tr.find_next('td').find_next('td').text.strip().split('years')[0]
            else:
                continue

        return player_info
    except:
        return {}


In [56]:
# def get_total_matches(soup):
#     try:
#         total_matches = soup.find('td', string="Matches:").find_next('td').text.split('(')[0].strip()
#     except AttributeError:
#         total_matches = -1
#     return total_matches

In [64]:
def get_player_stats(player_id):
    PLAYER_STAT = {}

    for url in URL:
        PLAYER_INFO_FLAG = False
        stats = {
            "batting": {},
            "bowling": {},
            "captaincy": {}
        }
        request_url = url + player_id
        response = requests.get(request_url, timeout=10)

        if response.status_code != 200:
            print("Player stats not found for player id " + player_id + " with response code " + str(response.status_code))
            if url == player_base_odi_url:
                PLAYER_STAT['odi'] = stats
            elif url == player_base_t20_url:
                PLAYER_STAT['t20'] = stats
            elif url == player_base_test_url:
                PLAYER_STAT['test'] = stats
            else:
                print("Unknown url " + url)
        else:
            soup = BeautifulSoup(response.content, 'html.parser')
            if not PLAYER_INFO_FLAG:
                PLAYER_INFO = get_player_info(soup)
                PLAYER_INFO_FLAG = True
                if PLAYER_INFO is None:
                    return None
                else:
                    PLAYER_STAT.update(PLAYER_INFO)
            if url == player_base_odi_url:
                PLAYER_STAT['odi'] = odi_t20_stat(soup)
                PLAYER_STAT['totalODIMatches'] = PLAYER_INFO.get('matches', -1)
                # PLAYER_STAT['totalODIMatches'] = get_total_matches(soup)
            elif url == player_base_t20_url:
                PLAYER_STAT['t20'] = odi_t20_stat(soup)
                PLAYER_STAT['totalT20Matches'] = get_total_matches(soup)
            elif url == player_base_test_url:
                PLAYER_STAT['test'] = test_stat(soup)
                PLAYER_STAT['totalTestMatches'] = get_total_matches(soup)
            else:
                print("Unknown url " + url)

    return PLAYER_STAT

In [65]:
def get_odi_stats(player_id):
    PLAYER_STAT = {}

    url = player_base_odi_url
    PLAYER_INFO_FLAG = False
    stats = {
        "batting": {},
        "bowling": {},
        "captaincy": {}
    }
    request_url = url + player_id
    response = requests.get(request_url, timeout=10)

    print(response.status_code)

    if response.status_code != 200:
        print("Player stats not found for player id " + player_id + " with response code " + str(response.status_code))
        if url == player_base_odi_url:
            PLAYER_STAT['odi'] = stats
        elif url == player_base_t20_url:
            PLAYER_STAT['t20'] = stats
        elif url == player_base_test_url:
            PLAYER_STAT['test'] = stats
        else:
            print("Unknown url " + url)
    else:
        soup = BeautifulSoup(response.content, 'html.parser')
        if not PLAYER_INFO_FLAG:
            PLAYER_INFO = get_player_info(soup)
            PLAYER_INFO_FLAG = True
            if PLAYER_INFO is None:
                return {}
            else:
                PLAYER_STAT.update(PLAYER_INFO)
        if url == player_base_odi_url:
            PLAYER_STAT['odi'] = odi_t20_stat(soup)
            PLAYER_STAT['totalODIMatches'] = PLAYER_INFO.get('matches', -1)
        elif url == player_base_t20_url:
            PLAYER_STAT['t20'] = odi_t20_stat(soup)
            # PLAYER_STAT['totalT20Matches'] = get_total_matches(soup)
        elif url == player_base_test_url:
            PLAYER_STAT['test'] = test_stat(soup)
            # PLAYER_STAT['totalTestMatches'] = get_total_matches(soup)
        else:
            print("Unknown url " + url)

    return PLAYER_STAT

In [66]:
all_player = []
import pandas as pd
import time
import random
data = pd.read_csv('player_id.csv')

In [67]:
for i in range(len(data)):
    player_id = str(data['howstatid'][i])
    stat = get_odi_stats(player_id)
    print(stat)
    stat['howstatid'] = player_id
    stat['name'] = data['name'][i]
    stat['role'] = data['role'][i]
    if stat is None:
        continue
    else:
        all_player.append(stat)

    time.sleep(random.randint(1, 15))

200
{'name': 'Arshdeep Singh', 'age': '24 ', 'matches': '3', 'odi': {'batting': {'innings': '1', 'notOuts': '0', 'agrregate': '9', 'highestScore': '9', 'average': '9.00', '50s': '0', '100s': '0', 'ducks': '0', '4s': '0', '6s': '1', 'scoringRate': '100.00', 'BallsFaced': '9'}, 'bowling': {'ballDeliveries': '79', 'runsConceded': '89', 'wicktes': '0', 'bowlingAverage': '9.00', '5Wickets': '0', 'bestBowling': '0/21', 'bowlingEconomyRate': '6.76', 'bowlingStrikeRate': 'N/A'}, 'captaincy': {'Matches/Won/Lost:': None}}, 'totalODIMatches': '3'}
200
{'name': 'Ravichandran Ashwin', 'age': '37 ', 'matches': '116', 'odi': {'batting': {'innings': '63', 'notOuts': '20', 'agrregate': '707', 'highestScore': '65', 'average': '16.44', '50s': '1', '100s': '0', 'ducks': '6', '4s': '60', '6s': '7', 'scoringRate': '86.96', 'BallsFaced': '813'}, 'bowling': {'ballDeliveries': '6303', 'runsConceded': '5180', 'wicktes': '156', 'bowlingAverage': '16.44', '5Wickets': '0', 'bestBowling': '4/25', 'bowlingEconomyRat

In [68]:
all_player

[{'name': 'Arshdeep Singh',
  'age': '24 ',
  'matches': '3',
  'odi': {'batting': {'innings': '1',
    'notOuts': '0',
    'agrregate': '9',
    'highestScore': '9',
    'average': '9.00',
    '50s': '0',
    '100s': '0',
    'ducks': '0',
    '4s': '0',
    '6s': '1',
    'scoringRate': '100.00',
    'BallsFaced': '9'},
   'bowling': {'ballDeliveries': '79',
    'runsConceded': '89',
    'wicktes': '0',
    'bowlingAverage': '9.00',
    '5Wickets': '0',
    'bestBowling': '0/21',
    'bowlingEconomyRate': '6.76',
    'bowlingStrikeRate': 'N/A'},
   'captaincy': {'Matches/Won/Lost:': None}},
  'totalODIMatches': '3',
  'howstatid': '4933',
  'role': 'bowler'},
 {'name': 'Ravichandran Ashwin',
  'age': '37 ',
  'matches': '116',
  'odi': {'batting': {'innings': '63',
    'notOuts': '20',
    'agrregate': '707',
    'highestScore': '65',
    'average': '16.44',
    '50s': '1',
    '100s': '0',
    'ducks': '6',
    '4s': '60',
    '6s': '7',
    'scoringRate': '86.96',
    'BallsFaced':

In [73]:
import pandas as pd
import time
import random
data2 = pd.read_csv('player_id.csv')

In [75]:
for p in range(len(all_player)):
    all_player[p]['role'] = data2['role'][p]

In [76]:
all_player

[{'name': 'Arshdeep Singh',
  'age': '24 ',
  'matches': '3',
  'odi': {'batting': {'innings': '1',
    'notOuts': '0',
    'agrregate': '9',
    'highestScore': '9',
    'average': '9.00',
    '50s': '0',
    '100s': '0',
    'ducks': '0',
    '4s': '0',
    '6s': '1',
    'scoringRate': '100.00',
    'BallsFaced': '9'},
   'bowling': {'ballDeliveries': '79',
    'runsConceded': '89',
    'wicktes': '0',
    'bowlingAverage': '9.00',
    '5Wickets': '0',
    'bestBowling': '0/21',
    'bowlingEconomyRate': '6.76',
    'bowlingStrikeRate': 'N/A'},
   'captaincy': {'Matches/Won/Lost:': None}},
  'totalODIMatches': '3',
  'howstatid': '4933',
  'role': 'bowler'},
 {'name': 'Ravichandran Ashwin',
  'age': '37 ',
  'matches': '116',
  'odi': {'batting': {'innings': '63',
    'notOuts': '20',
    'agrregate': '707',
    'highestScore': '65',
    'average': '16.44',
    '50s': '1',
    '100s': '0',
    'ducks': '6',
    '4s': '60',
    '6s': '7',
    'scoringRate': '86.96',
    'BallsFaced':

455

In [83]:
import json
with open('odi_stats.json', 'r') as f:
    data3 = json.load(f)

In [84]:
len(data3)

455

In [81]:
len(data2)

455

In [85]:
for p in range(len(data3)):
    data3[p]['country'] = data2['country'][p]

In [86]:
data3

[{'name': 'Arshdeep Singh',
  'age': '24 ',
  'matches': '3',
  'odi': {'batting': {'innings': '1',
    'notOuts': '0',
    'agrregate': '9',
    'highestScore': '9',
    'average': '9.00',
    '50s': '0',
    '100s': '0',
    'ducks': '0',
    '4s': '0',
    '6s': '1',
    'scoringRate': '100.00',
    'BallsFaced': '9'},
   'bowling': {'ballDeliveries': '79',
    'runsConceded': '89',
    'wicktes': '0',
    'bowlingAverage': '9.00',
    '5Wickets': '0',
    'bestBowling': '0/21',
    'bowlingEconomyRate': '6.76',
    'bowlingStrikeRate': 'N/A'},
   'captaincy': {'Matches/Won/Lost:': None}},
  'totalODIMatches': '3',
  'howstatId': '4933',
  'role': 'Bowler',
  'country': 'IND'},
 {'name': 'Ravichandran Ashwin',
  'age': '37 ',
  'matches': '116',
  'odi': {'batting': {'innings': '63',
    'notOuts': '20',
    'agrregate': '707',
    'highestScore': '65',
    'average': '16.44',
    '50s': '1',
    '100s': '0',
    'ducks': '6',
    '4s': '60',
    '6s': '7',
    'scoringRate': '86.96

In [87]:
json_data3 = json.dumps(data3)

In [88]:
json_data3

'[{"name": "Arshdeep Singh", "age": "24 ", "matches": "3", "odi": {"batting": {"innings": "1", "notOuts": "0", "agrregate": "9", "highestScore": "9", "average": "9.00", "50s": "0", "100s": "0", "ducks": "0", "4s": "0", "6s": "1", "scoringRate": "100.00", "BallsFaced": "9"}, "bowling": {"ballDeliveries": "79", "runsConceded": "89", "wicktes": "0", "bowlingAverage": "9.00", "5Wickets": "0", "bestBowling": "0/21", "bowlingEconomyRate": "6.76", "bowlingStrikeRate": "N/A"}, "captaincy": {"Matches/Won/Lost:": null}}, "totalODIMatches": "3", "howstatId": "4933", "role": "Bowler", "country": "IND"}, {"name": "Ravichandran Ashwin", "age": "37 ", "matches": "116", "odi": {"batting": {"innings": "63", "notOuts": "20", "agrregate": "707", "highestScore": "65", "average": "16.44", "50s": "1", "100s": "0", "ducks": "6", "4s": "60", "6s": "7", "scoringRate": "86.96", "BallsFaced": "813"}, "bowling": {"ballDeliveries": "6303", "runsConceded": "5180", "wicktes": "156", "bowlingAverage": "16.44", "5Wick